In [1]:
import numpy
import sys, os
from catboost import CatBoostRegressor, cv, Pool, CatBoostClassifier
import polars as pl
import pandas as pd

# import seaborn as sns
import matplotlib.pyplot as plt
import gc
import numpy as np
import joblib as jl
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import (
    MinMaxScaler,
    OrdinalEncoder,
    LabelBinarizer,
    OneHotEncoder,
    StandardScaler,
)
from sklearn.metrics import matthews_corrcoef

from sklearn.impute import SimpleImputer
import xgboost as xgb
from sklearn.model_selection import train_test_split
import random
from datetime import datetime

# helpers
sys.path.append("..")
from helpers.loss_functions import (
    update_submission,
    finalize_submission,
    handle_categorical_columns,
)

now = int(datetime.now().timestamp())
SEED = 108
random.seed(SEED)

N_FOLDS = 12
early_stop = 100
iterations = 10000
is_dshub = os.getenv("NB_USER", False)
if is_dshub:
    train_path = "../../data/train.csv"
    test_path = "../../data/test.csv"
else:
    train_path = "/home/manpm/Developers/kaggle/data/mushrooms/train.csv"
    test_path = "/home/manpm/Developers/kaggle/data/mushrooms/test.csv"

In [2]:
# Prepare data
train = pl.scan_parquet("../train_fold.parquet")
categorical_cols = train.select(pl.col(pl.Utf8)).columns
categorical_cols.pop(0)
gc.collect()
categorical_cols

/tmp/ipykernel_7595/1904636617.py:3: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  categorical_cols = train.select(pl.col(pl.Utf8)).columns


['cap-shape',
 'cap-surface',
 'cap-color',
 'does-bruise-or-bleed',
 'gill-attachment',
 'gill-spacing',
 'gill-color',
 'stem-root',
 'stem-surface',
 'stem-color',
 'veil-type',
 'veil-color',
 'has-ring',
 'ring-type',
 'spore-print-color',
 'habitat',
 'season']

## predict

In [3]:
# train_pool = Pool(
#     X_train_splitted,
#     label=y_train_splitted,
#     cat_features=categorical_training_cols,
# )
# val_pool = Pool(
#     X_test_splitted,
#     label=y_test_splitted,
#     cat_features=categorical_training_cols,
# )
params = {
    "loss_function": "Logloss",
    "iterations": iterations,
    "min_data_in_leaf": 15,
    "task_type": "GPU",
    "depth": 15,
    "l2_leaf_reg": 5.923260155898824,
    "learning_rate": 0.06953515790615505,
    "bagging_temperature": 0.270792508445483,
    "random_strength": 0.8745583257807371,
}

# model = CatBoostClassifier(**params)
# # train the model
# model.fit(
#     train_pool,
#     use_best_model=True,
#     eval_set=val_pool,
#     metric_period=100,
#     early_stopping_rounds=50,
# )
# y_pred = model.predict(val_pool)

In [4]:
y_preds = []
y_trues = []
for idx in range(N_FOLDS):
    X_train = (
        train.filter(pl.col("fold") != idx)
        .drop(["id", "class"])
        .fill_null("other")
        .drop(["fold"])
        .collect()
        .to_pandas()
    )
    for c in categorical_cols:
        value_counts = (
            X_train.groupby(c)
            .agg(pl.count().alias(f"{c}_count"))
            .sort(f"{c}_count", reverse=True)
            .select(c)
            .head(5)
        )
    y_train = (
        train.filter(pl.col("fold") != idx).select(pl.col("class")).collect().to_numpy()
    )

    X_val = (
        train.filter(pl.col("fold") == idx)
        .drop(["id", "class"])
        .fill_null("other")
        .drop(["fold"])
        .collect()
        .to_pandas()
    )
    y_val = (
        train.filter(pl.col("fold") == idx).select(pl.col("class")).collect().to_numpy()
    )

    train_pool = Pool(
        X_train,
        label=y_train,
        cat_features=categorical_cols,
    )
    val_pool = Pool(
        X_val,
        label=y_val,
        cat_features=categorical_cols,
    )

    model = CatBoostClassifier(**params)
    # train the model
    model.fit(
        train_pool,
        use_best_model=True,
        eval_set=val_pool,
        metric_period=50,
        early_stopping_rounds=early_stop,
        plot=True,
    )
    y_pred = model.predict(val_pool)
    y_preds.append(y_pred)
    y_trues.append(y_val)
    gc.collect()
# Concatenate the predictions and true labels
y_preds_concat = np.concatenate(y_preds)
y_trues_concat = np.concatenate(y_trues)
jl.dump(model, f"catboost_clf_{now}.pkl")

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.6595960	test: 0.6595790	best: 0.6595790 (0)	total: 537ms	remaining: 26.3s
49:	learn: 0.0432884	test: 0.0444094	best: 0.0444094 (49)	total: 26.1s	remaining: 0us
bestTest = 0.04440938526
bestIteration = 49


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.6603158	test: 0.6604556	best: 0.6604556 (0)	total: 530ms	remaining: 26s
49:	learn: 0.0431987	test: 0.0443903	best: 0.0443903 (49)	total: 26.3s	remaining: 0us
bestTest = 0.04439033302
bestIteration = 49


['catboost_clf_1724164216.pkl']

In [5]:
mcc = matthews_corrcoef(y_trues_concat, y_preds_concat)
print(f"Validation mcc score: {mcc}")

Validation mcc score: 0.9795768449188189


In [6]:
X_test = (
    pl.scan_csv(test_path)
    .drop(
        [
            "id",
        ]
    )
    .fill_null("other")
    .collect()
    .to_pandas()
)

In [7]:
update_submission(
    model=model,
    X_test=X_test,
    unique_target_values=["e", "p"],
    categorical_cols=categorical_cols,
)
finalize_submission(strategy="mean")